In [32]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html, callback
import base64
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)
img_src = "./Logo.png"
encoded_img = base64.b64encode(open(img_src, 'rb').read())


# Class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


app.layout = html.Div([
    html.H1("Project 2 - Phillip Cabaniss"),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_img.decode()), style={
                'height': 300,
                'width': 300,
            })),
    html.Hr(),
    html.Div(className = 'buttonRow',
             style={'display' : 'flex'},
             children=[
                 html.Button(id='waterRescue', n_clicks=0, children='Water Rescue'),
                 html.Button(id='mtnRescue', n_clicks=0, children='Mountain Rescue'),
                 html.Button(id='dstrRescue', n_clicks=0, children='Disaster Rescue'),
                 html.Button(id='resetButton', n_clicks=0, children='Reset'),
             ]
            ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,

    ),
    html.Br(),
     html.Hr(),
     html.Div(
         className='row',
         style={'display':'flex'},
         children= [
            html.Div(
            id='chart-id',
            className='col s12 m6',
            ),
            html.Div( 
            id='map-id',
            className='col s12 m6',
            ),
         ]),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')     
    ])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@callback(Output('tbl_out', 'children'), Input('tbl', 'active_cell'))
def update_graphs(active_cell):
    return str(active_cell) if active_cell else "Click the table"
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
 # Because we only allow single row selection, the list can 
 # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           dl.Marker(position=[dff.loc[row,'location_lat'],dff.loc[row,'location_long']],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]
@app.callback(
    Output('chart-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    ])
def update_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    
    return[
        dcc.Graph(
            figure = px.histogram(dff, x='breed')
        )]
    
# Callack for data buttons   
@app.callback(
    Output('datatable-id', "data"),
    [Input('waterRescue', 'n_clicks'),   
     Input('mtnRescue', 'n_clicks'),
     Input('dstrRescue', 'n_clicks'),
     Input('resetButton', 'n_clicks')
    ])
def on_click(button1, button2, button3, button4):
    #df = pd.DataFrame.from_records(shelter.read({}))
    
    # Water Rescue
    if (int(button1) > int(button2) and int(button1) > int(button3) and int(button1) > int(button4)):
        df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                                 ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}}))
    # Mountain Rescue
    elif (int(button2) > int(button1) and int(button2) > int(button3) and int(button2) > int(button4)):
        df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                                  "Siberian Husky", "Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}}))
    # Disaster Rescue
    elif (int(button3) > int(button1) and int(button3) > int(button2) and int(button3) > int(button4)):
        df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                                  "Bloodhound","Rottweiler"
                                 ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}}))
    # Reset
    elif (int(button4) > int(button1) and int(button4) > int(button2) and int(button4) > int(button3)):
        df = pd.DataFrame.from_records(shelter.read({}))
    
    #df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

del app.config._read_only["requests_pathname_prefix"]
app.run_server(debug=True)

Successfully connected...
Dash app running on http://127.0.0.1:20156/


/home/phillipcabani_snhu/.local/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

